# Limpieza de datos del dataset de coches de segunda mano de Kaggle.

## 1. Información de los datos Scrapeados.

En este Notebook se limpian los datos del dataset de coches de segunda mano de kaggle que se usará en caso de no contar con suficientes vehículos scrapeados, para crear un mejor modelo.

Por ello, los datos presentes en este dataset se transformarán levemente para que tengan las mismas features y formato que los datos extraídos mediante el WebScraping que se ha realizado de [coches.net](coches.net).

Lo primero que haremos es cargar algunos vehículos de la tabla TX_VEHICULOS_SEG_MANO de la base de datos en la que guardamos los vehículos scrapeados para ver los campos de la tabla y su formato:

In [67]:
import pandas as pd
import sqlite3

# Conexión a la BBDD:
con = sqlite3.connect("../include/db_vehiculos_inicial_webscraping.db")

# Vamos a ver qué nos devuelve un select sobre la tabla:
query = """
SELECT *
FROM TX_VEHICULOS_SEG_MANO
where
    origen_anuncio = 'WebScraping'
    --and marca not in ('BMW', 'VOLKSWAGEN', 'MERCEDES-BENZ', 'AUDI', 'PEUGEOT', 'FORD', 'RENAULT', 'OPEL', 'CITROEN', 'SEAT')
order by
    fecha_carga desc
;
"""

# Leemos los resultados en un DataFrame de pandas
df_db = pd.read_sql_query(query, con)

# Mostramos los primeros registros:
df_db.head()

,pk_anuncio_id,marca,modelo,precio,combustible,anyo_vehiculo,kilometraje,potencia,num_puertas,num_plazas,tipo_cambio,tipo_vehiculo,cilindrada_motor,color,provincia,etiqueta_eco,origen_anuncio,fecha_publicacion,anyomes_publicacion,fecha_carga,url
0,60579674,OPEL,Corsa 1.2 XEL Edition,12290.0,Gasolina,2022,58168,75.0,5,5.0,Manual,Berlina,1199.0,Rojo,Valencia,C (verde),WebScraping,2025-05-16,202505.0,2025-05-16 15:58:23,https://www.coches.net/opel-corsa-12-xel-55kw-...
1,60579673,OPEL,Corsa 1.2T XHL GSLine,14510.0,Gasolina,2022,18652,100.0,5,5.0,Manual,Berlina,1199.0,Azul,Valencia,C (verde),WebScraping,2025-05-16,202505.0,2025-05-16 15:57:56,https://www.coches.net/opel-corsa-12t-xhl-74kw...
2,60579672,OPEL,Corsa 1.2T XHL GSLine,13950.0,Gasolina,2022,27493,100.0,5,5.0,Manual,Berlina,1199.0,Gris / Plata,Valencia,C (verde),WebScraping,2025-05-16,202505.0,2025-05-16 15:57:34,https://www.coches.net/opel-corsa-12t-xhl-74kw...
3,60579678,OPEL,Corsa 1.2 XEL Edition,12290.0,Gasolina,2022,49466,75.0,5,5.0,Manual,Berlina,1199.0,Blanco,Valencia,C (verde),WebScraping,2025-05-16,202505.0,2025-05-16 15:56:49,https://www.coches.net/opel-corsa-12-xel-55kw-...
4,60580266,SEAT,Ibiza 1.6 TDI Style,11499.0,Diesel,2018,147508,95.0,5,5.0,Manual,Berlina,1598.0,Blanco,Valencia,C (verde),WebScraping,2025-05-16,202505.0,2025-05-16 15:56:26,https://www.coches.net/seat-ibiza-16-tdi-70kw-...


In [68]:
df_db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731 entries, 0 to 730
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   pk_anuncio_id        731 non-null    int64  
 1   marca                731 non-null    object 
 2   modelo               731 non-null    object 
 3   precio               731 non-null    float64
 4   combustible          718 non-null    object 
 5   anyo_vehiculo        731 non-null    int64  
 6   kilometraje          731 non-null    int64  
 7   potencia             728 non-null    float64
 8   num_puertas          731 non-null    int64  
 9   num_plazas           728 non-null    float64
 10  tipo_cambio          731 non-null    object 
 11  tipo_vehiculo        731 non-null    object 
 12  cilindrada_motor     719 non-null    float64
 13  color                722 non-null    object 
 14  provincia            731 non-null    object 
 15  etiqueta_eco         564 non-null    obj

Vemos que la tabla TX_VEHICULOS_SEG_MANO tiene 21 columnas, en la que cada registro se identifica únicamente por el pk_anuncio_id.

Además, tiene varias columnas tanto numéricas como categóricas que servirán para features del modelo predictivo, la columna precio será el target a predecir. 

También existe una columna llamada ```"origen_anuncio"``` que nos servirá para distinguir entre anuncios scrapeados o provenientes del dataset de Kaggle en caso de combinar ambos orígenes de datos y, por último, hay columnas informativas como las columnas: ```["fecha_publicacion", "anyomes_publicacion", "fecha_carga", "url"]```.

## 2. Información de los datos del Dataset de Kaggle.

Ahora vamos a ver el tipo de datos que contiene el dataset de Kaggle.

Importamos los datos desde el siguiente link:

[https://www.kaggle.com/datasets/datamarket/venta-de-coches](https://www.kaggle.com/datasets/datamarket/venta-de-coches)

Creo que el link no funciona actualmente, por lo que importaremos los datos desde local, ya que los tengo descargados.

In [44]:
# Importamos las librerías necesarias

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [45]:
df = pd.read_csv("../include/data/raw/coches-de-segunda-mano-sample.csv")
df.head()

,url,company,make,model,version,price,price_financed,fuel,year,kms,...,doors,shift,color,photos,is_professional,dealer,province,country,publish_date,insert_date
0,e158ae0ca53119ca199c28c36b5c2fcd,9881bcdd5a0ad4733037b3fb25e69c3a,SEAT,Toledo,SEAT Toledo 4p.,950,NaN,Diésel,2000.0,227000,...,4,Manual,Verde,5,False,0f4bb8455d27349b8273109b66a847f3,Navarra,Spain,2020-12-18 10:47:13,2021-01-15 00:00:00
1,ff267ebb7e700246f47f84f3db660b4b,9881bcdd5a0ad4733037b3fb25e69c3a,CITROEN,C1,CITROEN C1 PureTech 60KW 82CV Feel 5p.,6200,NaN,Gasolina,2017.0,50071,...,5,Manual,Blanco,6,True,Autos Raymara,Tenerife,Spain,2021-01-02 11:25:40,2021-01-15 00:00:00
2,de4b02db28ea7786c622b969be10c7c7,9881bcdd5a0ad4733037b3fb25e69c3a,FORD,Transit Connect,FORD Transit Connect Van 1.5 TDCi 100cv Ambien...,7851,7024.0,Diésel,2016.0,103000,...,4,Manual,Blanco,10,True,Auto 96,Barcelona,Spain,2020-12-16 10:51:45,2021-01-15 00:00:00
3,0449972a4d07594acf92e9a7dd28b39c,9881bcdd5a0ad4733037b3fb25e69c3a,VOLKSWAGEN,Caravelle,VOLKSWAGEN Caravelle Largo 2.0 TDI 140 Comfort...,19426,NaN,Diésel,2014.0,120000,...,4,Manual,Blanco,9,True,Inniauto,Navarra,Spain,2020-11-25 11:09:14,2021-01-15 00:00:00
4,12c4fa49bd4fdf23f19ecf396d3f02ef,9881bcdd5a0ad4733037b3fb25e69c3a,FORD,Transit,FORD Transit 350 96kW L4 Ambiente Propulsion T...,22850,22800.0,Diésel,2017.0,107000,...,2,Manual,Blanco,4,True,"Autofleet España,s.l",Sevilla,Spain,2021-01-12 20:00:34,2021-01-15 00:00:00


In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   url              50000 non-null  object 
 1   company          50000 non-null  object 
 2   make             49998 non-null  object 
 3   model            49995 non-null  object 
 4   version          50000 non-null  object 
 5   price            50000 non-null  int64  
 6   price_financed   23563 non-null  float64
 7   fuel             49954 non-null  object 
 8   year             49998 non-null  float64
 9   kms              50000 non-null  int64  
 10  power            41472 non-null  float64
 11  doors            50000 non-null  int64  
 12  shift            49889 non-null  object 
 13  color            49406 non-null  object 
 14  photos           50000 non-null  int64  
 15  is_professional  50000 non-null  bool   
 16  dealer           50000 non-null  object 
 17  province    

Vemos que el Dataset de Kaggle cuenta con 50.000 registros. Vamos a ver cuántos registros tiene por marca:

In [47]:
# Número de coches por marca en el df:

num_coches_marca_df = df.groupby("make")["make"].count().sort_values(ascending=False)

pd.set_option("display.max_rows", None)
print(num_coches_marca_df)

make
VOLKSWAGEN        4624
BMW               4510
MERCEDES-BENZ     4227
AUDI              3745
PEUGEOT           3314
RENAULT           3127
FORD              3116
OPEL              2924
CITROEN           2855
SEAT              2604
NISSAN            1634
TOYOTA            1562
FIAT              1398
HYUNDAI           1149
KIA               1118
VOLVO              823
MINI               811
LAND-ROVER         595
SKODA              574
PORSCHE            517
MAZDA              490
HONDA              464
JAGUAR             339
MITSUBISHI         334
ALFA ROMEO         302
JEEP               299
LEXUS              282
DACIA              277
SUZUKI             252
CHEVROLET          224
SSANGYONG          216
SMART              196
CHRYSLER           126
DS                 125
SUBARU              99
ABARTH              96
INFINITI            78
SAAB                73
LANCIA              62
MASERATI            62
IVECO               48
DAEWOO              45
ROVER               36
FERRAR

Observamos que para la marca que más vehículos hay registrados es ```VolksWagen```, con 4.624 registros.

Como solo estamos scrapeando vehículos de las siguientes 10 marcas (ya que tampoco se pueden scrapear más rápido sin ser detectados y para que podamos realizar un modelo que funcione bien para estas 10), nos vamos a quedar sólo con los vehículos de estas marcas:

```['BMW', 'VOLKSWAGEN', 'MERCEDES-BENZ', 'AUDI', 'PEUGEOT', 'FORD', 'RENAULT', 'OPEL', 'CITROEN', 'SEAT']```

In [48]:
df = df.loc[df["make"].isin(['BMW', 'VOLKSWAGEN', 'MERCEDES-BENZ', 'AUDI', 'PEUGEOT', 
                        'FORD', 'RENAULT', 'OPEL', 'CITROEN', 'SEAT'])]

df.head()

,url,company,make,model,version,price,price_financed,fuel,year,kms,...,doors,shift,color,photos,is_professional,dealer,province,country,publish_date,insert_date
0,e158ae0ca53119ca199c28c36b5c2fcd,9881bcdd5a0ad4733037b3fb25e69c3a,SEAT,Toledo,SEAT Toledo 4p.,950,NaN,Diésel,2000.0,227000,...,4,Manual,Verde,5,False,0f4bb8455d27349b8273109b66a847f3,Navarra,Spain,2020-12-18 10:47:13,2021-01-15 00:00:00
1,ff267ebb7e700246f47f84f3db660b4b,9881bcdd5a0ad4733037b3fb25e69c3a,CITROEN,C1,CITROEN C1 PureTech 60KW 82CV Feel 5p.,6200,NaN,Gasolina,2017.0,50071,...,5,Manual,Blanco,6,True,Autos Raymara,Tenerife,Spain,2021-01-02 11:25:40,2021-01-15 00:00:00
2,de4b02db28ea7786c622b969be10c7c7,9881bcdd5a0ad4733037b3fb25e69c3a,FORD,Transit Connect,FORD Transit Connect Van 1.5 TDCi 100cv Ambien...,7851,7024.0,Diésel,2016.0,103000,...,4,Manual,Blanco,10,True,Auto 96,Barcelona,Spain,2020-12-16 10:51:45,2021-01-15 00:00:00
3,0449972a4d07594acf92e9a7dd28b39c,9881bcdd5a0ad4733037b3fb25e69c3a,VOLKSWAGEN,Caravelle,VOLKSWAGEN Caravelle Largo 2.0 TDI 140 Comfort...,19426,NaN,Diésel,2014.0,120000,...,4,Manual,Blanco,9,True,Inniauto,Navarra,Spain,2020-11-25 11:09:14,2021-01-15 00:00:00
4,12c4fa49bd4fdf23f19ecf396d3f02ef,9881bcdd5a0ad4733037b3fb25e69c3a,FORD,Transit,FORD Transit 350 96kW L4 Ambiente Propulsion T...,22850,22800.0,Diésel,2017.0,107000,...,2,Manual,Blanco,4,True,"Autofleet España,s.l",Sevilla,Spain,2021-01-12 20:00:34,2021-01-15 00:00:00


In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 35046 entries, 0 to 49999
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   url              35046 non-null  object 
 1   company          35046 non-null  object 
 2   make             35046 non-null  object 
 3   model            35041 non-null  object 
 4   version          35046 non-null  object 
 5   price            35046 non-null  int64  
 6   price_financed   15636 non-null  float64
 7   fuel             35028 non-null  object 
 8   year             35045 non-null  float64
 9   kms              35046 non-null  int64  
 10  power            28266 non-null  float64
 11  doors            35046 non-null  int64  
 12  shift            34966 non-null  object 
 13  color            34601 non-null  object 
 14  photos           35046 non-null  int64  
 15  is_professional  35046 non-null  bool   
 16  dealer           35046 non-null  object 
 17  province         

In [50]:
# Comprobamos el número de marcas ahora:

num_coches_marca_df = df.groupby("make")["make"].count().sort_values(ascending=False)

pd.set_option("display.max_rows", None)
print(num_coches_marca_df)

make
VOLKSWAGEN       4624
BMW              4510
MERCEDES-BENZ    4227
AUDI             3745
PEUGEOT          3314
RENAULT          3127
FORD             3116
OPEL             2924
CITROEN          2855
SEAT             2604
Name: make, dtype: int64


Nos hemos quedado con **35.046 registros, pertenecientes a 10 marcas**.

Si comparamos los campos presentes en ambos DataFrames, vemos que coinciden varios de ellos. Vamos a **eliminar** los siguientes cambios de este DataFrame importado de Kaggle, ya que no existen en los registros scrapeados:

- ```company```
- ```version```
- ```price_financed```
- ```photos```
- ```is_professional```
- ```dealer```
- ```country```

In [51]:
df.drop(columns=["company", "version", "price_financed", "photos", "is_professional", "dealer", "country"],
         inplace=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 35046 entries, 0 to 49999
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   url           35046 non-null  object 
 1   make          35046 non-null  object 
 2   model         35041 non-null  object 
 3   price         35046 non-null  int64  
 4   fuel          35028 non-null  object 
 5   year          35045 non-null  float64
 6   kms           35046 non-null  int64  
 7   power         28266 non-null  float64
 8   doors         35046 non-null  int64  
 9   shift         34966 non-null  object 
 10  color         34601 non-null  object 
 11  province      35042 non-null  object 
 12  publish_date  35046 non-null  object 
 13  insert_date   35046 non-null  object 
dtypes: float64(2), int64(3), object(9)
memory usage: 4.0+ MB


Vemos que ahora nos quedamos con 15 columnas.

Además, también vamos a renombrar los siguientes campos para que coincidan con los de nuestra tabla original:

- ```make``` --> ```marca```
- ```model``` --> ```modelo```
- ```price``` --> ```precio```
- ```fuel``` --> ```combustible```
- ```year``` --> ```anyo_vehiculo```
- ```kms``` --> ```kilometraje```
- ```power``` --> ```potencia```
- ```doors``` --> ```num_puertas```
- ```shift``` --> ```tipo_cambio```
- ```province``` --> ```provincia```
- ```publish_date``` --> ```fecha_publicacion```
- ```insert_date``` --> ```fecha_carga```

También creamos los siguientes campos para poder insertar estos registros en la base de datos:

- ```pk_anuncio_id```: Secuencial que identifica el vehículo. No coincide con el de los vehículos scrapeados, ya tenemos 35.046 en este dataset y los scrapeados empiezan en 50000000.
- ```origen_anuncio```: Tendrá el valor "dataset kaggle" para todos estos registros importados desde el Dataset.
- ```anyomes_publicacion```: Año y mes de la fecha de publicación en formato 'YYYYMM'.

Creamos también los siguientes campos, aunque los dejaremos como *null* para los datos de este origen:

- ```num_plazas```: *null*
- ```tipo_vehiculo```: *null*
- ```cilindrada_motor```: *null*
- ```etiqueta_eco```: *null*

In [53]:
import pandas as pd

# Renombramos las columnas:
df = df.rename(columns={
    'make': 'marca',
    'model': 'modelo',
    'price': 'precio',
    'fuel': 'combustible',
    'year': 'anyo_vehiculo',
    'kms': 'kilometraje',
    'power': 'potencia',
    'doors': 'num_puertas',
    'shift': 'tipo_cambio',
    'province': 'provincia',
    'publish_date': 'fecha_publicacion',
    'insert_date': 'fecha_carga'
})

# Campo 'pk_anuncio_id':
df['pk_anuncio_id'] = range(1, len(df) + 1)

# Campo 'origen_anuncio':
df['origen_anuncio'] = 'dataset kaggle'

# Campo 'anyomes_publicacion' en formato YYYYMM:
df['fecha_publicacion'] = pd.to_datetime(df['fecha_publicacion'], errors='coerce')
df['anyomes_publicacion'] = df['fecha_publicacion'].dt.strftime('%Y%m')

# Campos nulos:
df['num_plazas'] = None
df['tipo_vehiculo'] = None
df['cilindrada_motor'] = None
df['etiqueta_eco'] = None

# Conversión de tipos:
df['fecha_publicacion'] = df['fecha_publicacion'].dt.strftime('%Y-%m-%d')
df['anyomes_publicacion'] = df['anyomes_publicacion'].astype('Int64')  # Permite nulos
df['precio'] = df['precio'].astype(float)

# Reordenamos las columnas:
orden_columnas = [
    'pk_anuncio_id',
    'marca',
    'modelo',
    'precio',
    'combustible',
    'anyo_vehiculo',
    'kilometraje',
    'potencia',
    'num_puertas',
    'num_plazas',
    'tipo_cambio',
    'tipo_vehiculo',
    'cilindrada_motor',
    'color',
    'provincia',
    'etiqueta_eco',
    'origen_anuncio',
    'fecha_publicacion',
    'anyomes_publicacion',
    'fecha_carga',
    'url'
]

df = df[orden_columnas]


df.head()

,pk_anuncio_id,marca,modelo,precio,combustible,anyo_vehiculo,kilometraje,potencia,num_puertas,num_plazas,...,tipo_vehiculo,cilindrada_motor,color,provincia,etiqueta_eco,origen_anuncio,fecha_publicacion,anyomes_publicacion,fecha_carga,url
0,1,SEAT,Toledo,950.0,Diésel,2000.0,227000,NaN,4,None,...,None,None,Verde,Navarra,None,dataset kaggle,2020-12-18,202012,2021-01-15 00:00:00,e158ae0ca53119ca199c28c36b5c2fcd
1,2,CITROEN,C1,6200.0,Gasolina,2017.0,50071,82.0,5,None,...,None,None,Blanco,Tenerife,None,dataset kaggle,2021-01-02,202101,2021-01-15 00:00:00,ff267ebb7e700246f47f84f3db660b4b
2,3,FORD,Transit Connect,7851.0,Diésel,2016.0,103000,100.0,4,None,...,None,None,Blanco,Barcelona,None,dataset kaggle,2020-12-16,202012,2021-01-15 00:00:00,de4b02db28ea7786c622b969be10c7c7
3,4,VOLKSWAGEN,Caravelle,19426.0,Diésel,2014.0,120000,140.0,4,None,...,None,None,Blanco,Navarra,None,dataset kaggle,2020-11-25,202011,2021-01-15 00:00:00,0449972a4d07594acf92e9a7dd28b39c
4,5,FORD,Transit,22850.0,Diésel,2017.0,107000,130.0,2,None,...,None,None,Blanco,Sevilla,None,dataset kaggle,2021-01-12,202101,2021-01-15 00:00:00,12c4fa49bd4fdf23f19ecf396d3f02ef


In [54]:
# Comprobamos que el id se genera bien:

#df["pk_anuncio_id"].value_counts()

Comprobamos como ha quedado el df asignado al Dataset de Kaggle:

In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 35046 entries, 0 to 49999
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   pk_anuncio_id        35046 non-null  int64  
 1   marca                35046 non-null  object 
 2   modelo               35041 non-null  object 
 3   precio               35046 non-null  float64
 4   combustible          35028 non-null  object 
 5   anyo_vehiculo        35045 non-null  float64
 6   kilometraje          35046 non-null  int64  
 7   potencia             28266 non-null  float64
 8   num_puertas          35046 non-null  int64  
 9   num_plazas           0 non-null      object 
 10  tipo_cambio          34966 non-null  object 
 11  tipo_vehiculo        0 non-null      object 
 12  cilindrada_motor     0 non-null      object 
 13  color                34601 non-null  object 
 14  provincia            35042 non-null  object 
 15  etiqueta_eco         0 non-null      obje

Ahora que ya tenemos las columnas que equivalen a las que obtenemos mediante WebScraping, vamos a compararlas, ya que todavía hay modificaciones que realizar para poder combinar ambas fuentes de datos.

Recuperamos las columnas del ```df_db```, que contiene los vehículos del WebScraping:

In [56]:
df_db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731 entries, 0 to 730
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   pk_anuncio_id        731 non-null    int64  
 1   marca                731 non-null    object 
 2   modelo               731 non-null    object 
 3   precio               731 non-null    float64
 4   combustible          718 non-null    object 
 5   anyo_vehiculo        731 non-null    int64  
 6   kilometraje          731 non-null    int64  
 7   potencia             728 non-null    float64
 8   num_puertas          731 non-null    int64  
 9   num_plazas           728 non-null    float64
 10  tipo_cambio          731 non-null    object 
 11  tipo_vehiculo        731 non-null    object 
 12  cilindrada_motor     719 non-null    float64
 13  color                722 non-null    object 
 14  provincia            731 non-null    object 
 15  etiqueta_eco         564 non-null    obj

In [ ]:
# Comprobamos el número de modelos ahora:

num_coches_modelo_df = df.groupby("modelo")["modelo"].count().sort_values(ascending=False)

pd.set_option("display.max_rows", None)
print(num_coches_modelo_df)

modelo
Golf                     1542
Serie 3                  1377
A4                        946
A3                        865
Focus                     854
Clase C                   847
Mégane                    835
Ibiza                     824
Astra                     812
León                      800
Serie 1                   731
Passat                    727
Serie 5                   567
Corsa                     556
Clio                      542
308                       540
Clase A                   504
Polo                      503
Clase E                   489
Fiesta                    469
C3                        376
Insignia                  370
Mondeo                    358
A6                        358
C4                        337
Touran                    310
3008                      286
A5                        283
Berlingo                  283
X1                        275
Tiguan                    262
208                       261
Scénic                    257
200

## Transformaciones para homogeneizar los datos de ambos orígenes.

Ahora vamos a ver las diferencias entre columnas de ambos DataFrames, para corregirlas y que queden lo más homogéneas posibles en la base de datos.

Coches dataset kaggle:

In [62]:
pd.set_option('display.max_columns', None)
df.head()

,pk_anuncio_id,marca,modelo,precio,combustible,anyo_vehiculo,kilometraje,potencia,num_puertas,num_plazas,tipo_cambio,tipo_vehiculo,cilindrada_motor,color,provincia,etiqueta_eco,origen_anuncio,fecha_publicacion,anyomes_publicacion,fecha_carga,url
0,1,SEAT,Toledo,950.0,Diésel,2000.0,227000,NaN,4,None,Manual,None,None,Verde,Navarra,None,dataset kaggle,2020-12-18,202012,2021-01-15 00:00:00,e158ae0ca53119ca199c28c36b5c2fcd
1,2,CITROEN,C1,6200.0,Gasolina,2017.0,50071,82.0,5,None,Manual,None,None,Blanco,Tenerife,None,dataset kaggle,2021-01-02,202101,2021-01-15 00:00:00,ff267ebb7e700246f47f84f3db660b4b
2,3,FORD,Transit Connect,7851.0,Diésel,2016.0,103000,100.0,4,None,Manual,None,None,Blanco,Barcelona,None,dataset kaggle,2020-12-16,202012,2021-01-15 00:00:00,de4b02db28ea7786c622b969be10c7c7
3,4,VOLKSWAGEN,Caravelle,19426.0,Diésel,2014.0,120000,140.0,4,None,Manual,None,None,Blanco,Navarra,None,dataset kaggle,2020-11-25,202011,2021-01-15 00:00:00,0449972a4d07594acf92e9a7dd28b39c
4,5,FORD,Transit,22850.0,Diésel,2017.0,107000,130.0,2,None,Manual,None,None,Blanco,Sevilla,None,dataset kaggle,2021-01-12,202101,2021-01-15 00:00:00,12c4fa49bd4fdf23f19ecf396d3f02ef


Coches WebScraping coches.net:

In [63]:
pd.set_option('display.max_columns', None)
df_db.head()

,pk_anuncio_id,marca,modelo,precio,combustible,anyo_vehiculo,kilometraje,potencia,num_puertas,num_plazas,tipo_cambio,tipo_vehiculo,cilindrada_motor,color,provincia,etiqueta_eco,origen_anuncio,fecha_publicacion,anyomes_publicacion,fecha_carga,url
0,60579674,OPEL,Corsa 1.2 XEL Edition,12290.0,Gasolina,2022,58168,75.0,5,5.0,Manual,Berlina,1199.0,Rojo,Valencia,C (verde),WebScraping,2025-05-16,202505.0,2025-05-16 15:58:23,https://www.coches.net/opel-corsa-12-xel-55kw-...
1,60579673,OPEL,Corsa 1.2T XHL GSLine,14510.0,Gasolina,2022,18652,100.0,5,5.0,Manual,Berlina,1199.0,Azul,Valencia,C (verde),WebScraping,2025-05-16,202505.0,2025-05-16 15:57:56,https://www.coches.net/opel-corsa-12t-xhl-74kw...
2,60579672,OPEL,Corsa 1.2T XHL GSLine,13950.0,Gasolina,2022,27493,100.0,5,5.0,Manual,Berlina,1199.0,Gris / Plata,Valencia,C (verde),WebScraping,2025-05-16,202505.0,2025-05-16 15:57:34,https://www.coches.net/opel-corsa-12t-xhl-74kw...
3,60579678,OPEL,Corsa 1.2 XEL Edition,12290.0,Gasolina,2022,49466,75.0,5,5.0,Manual,Berlina,1199.0,Blanco,Valencia,C (verde),WebScraping,2025-05-16,202505.0,2025-05-16 15:56:49,https://www.coches.net/opel-corsa-12-xel-55kw-...
4,60580266,SEAT,Ibiza 1.6 TDI Style,11499.0,Diesel,2018,147508,95.0,5,5.0,Manual,Berlina,1598.0,Blanco,Valencia,C (verde),WebScraping,2025-05-16,202505.0,2025-05-16 15:56:26,https://www.coches.net/seat-ibiza-16-tdi-70kw-...


Comparamos primero los valores de la columna ```"combustible"```:

In [60]:
df["combustible"].value_counts()

combustible
Diésel                23698
Gasolina              10468
Híbrido enchufable      315
Eléctrico               291
Híbrido                 149
Gas natural (CNG)        58
Gas licuado (GLP)        49
Name: count, dtype: int64

In [61]:
df_db["combustible"].value_counts()

combustible
Diesel                343
Gasolina              324
Híbrido                36
Eléctrico               9
Híbrido Enchufable      6
Name: count, dtype: int64

Vemos que son ligeramente distintas. Lo que haremos será corregir la carga del WebScraping para ajustarlos a los valores del dataset de kaggle.

Vamos con los valores de la columna ```"color"```:

In [64]:
df["color"].value_counts()

color
Gris / Plata                                                                                            7598
Blanco                                                                                                  7345
Negro                                                                                                   5802
Azul                                                                                                    3910
Rojo                                                                                                    1818
Verde                                                                                                    525
Marrón                                                                                                   466
Beige                                                                                                    464
Granate                                                                                                  360
Amarillo     

In [65]:
df_db["color"].value_counts()

color
Blanco          281
Gris / Plata    133
Negro           114
Azul             90
Rojo             79
Naranja           9
Marrón            5
Beige             5
Verde             3
Amarillo          3
Name: count, dtype: int64

Aquí vemos que los colores del dataset de kaggle son muy dispares. De momento, lo que haremos es quedarnos con la primera palabra en cada string y ponerlo en minúscula en ambos DataFrames:

In [ ]:
# Dataset kaggle:

import numpy as np

df["color"] = df["color"].str.split().str[0].str.lower()

# Reemplazamos con NaN si no empieza por una letra o número:
df["color"] = df["color"].where(df["color"].str.match(r'^[a-z0-9]', na=False), np.nan)


df["color"].value_counts()

color
gris             9081
blanco           8459
negro            6414
azul             4526
rojo             2113
verde             542
marrón            535
beige             498
granate           373
amarillo          287
naranja           204
300               104
475                73
violeta            55
2t2t               54
b4b4               51
x3x3               34
2y2y               33
0e0e               31
0q0q               31
b39                29
668                28
a83                25
8e8e               24
gaz                23
149                22
k5k5               21
h7h7               21
369                21
a89                19
0c0c               18
c10                18
l5l5               15
a90                15
a96                15
m0f4               14
y1y1               14
197                13
4m4m               13
b45                12
s7s7               12
z1z1               11
qpa$369            11
e1e1               10
p0wp               10
w1w1

In [ ]:
# Dataset WebScraping:

import numpy as np

df_db["color"] = df_db["color"].str.split().str[0].str.lower()

# Reemplazamos con NaN si no empieza por una letra o número:
df_db["color"] = df_db["color"].where(df_db["color"].str.match(r'^[a-z0-9]', na=False), np.nan)


df_db["color"].value_counts()

color
blanco      281
gris        133
negro       114
azul         90
rojo         79
naranja       9
marrón        5
beige         5
verde         3
amarillo      3
Name: count, dtype: int64

De momento vamos a dejarlo así, aunque quizás hagamos alguna transformación más para el modelo para los colores de los datos del Dataset de Kaggle.

Vamos a printear de nuevo los DataFrames a ver si hay alguna columna más que modificar:

In [72]:
#Dataset Kaggle:

pd.set_option('display.max_columns', None)
df.head()

,pk_anuncio_id,marca,modelo,precio,combustible,anyo_vehiculo,kilometraje,potencia,num_puertas,num_plazas,tipo_cambio,tipo_vehiculo,cilindrada_motor,color,provincia,etiqueta_eco,origen_anuncio,fecha_publicacion,anyomes_publicacion,fecha_carga,url
0,1,SEAT,Toledo,950.0,Diésel,2000.0,227000,NaN,4,None,Manual,None,None,verde,Navarra,None,dataset kaggle,2020-12-18,202012,2021-01-15 00:00:00,e158ae0ca53119ca199c28c36b5c2fcd
1,2,CITROEN,C1,6200.0,Gasolina,2017.0,50071,82.0,5,None,Manual,None,None,blanco,Tenerife,None,dataset kaggle,2021-01-02,202101,2021-01-15 00:00:00,ff267ebb7e700246f47f84f3db660b4b
2,3,FORD,Transit Connect,7851.0,Diésel,2016.0,103000,100.0,4,None,Manual,None,None,blanco,Barcelona,None,dataset kaggle,2020-12-16,202012,2021-01-15 00:00:00,de4b02db28ea7786c622b969be10c7c7
3,4,VOLKSWAGEN,Caravelle,19426.0,Diésel,2014.0,120000,140.0,4,None,Manual,None,None,blanco,Navarra,None,dataset kaggle,2020-11-25,202011,2021-01-15 00:00:00,0449972a4d07594acf92e9a7dd28b39c
4,5,FORD,Transit,22850.0,Diésel,2017.0,107000,130.0,2,None,Manual,None,None,blanco,Sevilla,None,dataset kaggle,2021-01-12,202101,2021-01-15 00:00:00,12c4fa49bd4fdf23f19ecf396d3f02ef


In [73]:
# Dataset WebScraping:

pd.set_option('display.max_columns', None)
df_db.head()

,pk_anuncio_id,marca,modelo,precio,combustible,anyo_vehiculo,kilometraje,potencia,num_puertas,num_plazas,tipo_cambio,tipo_vehiculo,cilindrada_motor,color,provincia,etiqueta_eco,origen_anuncio,fecha_publicacion,anyomes_publicacion,fecha_carga,url
0,60579674,OPEL,Corsa 1.2 XEL Edition,12290.0,Gasolina,2022,58168,75.0,5,5.0,Manual,Berlina,1199.0,rojo,Valencia,C (verde),WebScraping,2025-05-16,202505.0,2025-05-16 15:58:23,https://www.coches.net/opel-corsa-12-xel-55kw-...
1,60579673,OPEL,Corsa 1.2T XHL GSLine,14510.0,Gasolina,2022,18652,100.0,5,5.0,Manual,Berlina,1199.0,azul,Valencia,C (verde),WebScraping,2025-05-16,202505.0,2025-05-16 15:57:56,https://www.coches.net/opel-corsa-12t-xhl-74kw...
2,60579672,OPEL,Corsa 1.2T XHL GSLine,13950.0,Gasolina,2022,27493,100.0,5,5.0,Manual,Berlina,1199.0,gris,Valencia,C (verde),WebScraping,2025-05-16,202505.0,2025-05-16 15:57:34,https://www.coches.net/opel-corsa-12t-xhl-74kw...
3,60579678,OPEL,Corsa 1.2 XEL Edition,12290.0,Gasolina,2022,49466,75.0,5,5.0,Manual,Berlina,1199.0,blanco,Valencia,C (verde),WebScraping,2025-05-16,202505.0,2025-05-16 15:56:49,https://www.coches.net/opel-corsa-12-xel-55kw-...
4,60580266,SEAT,Ibiza 1.6 TDI Style,11499.0,Diesel,2018,147508,95.0,5,5.0,Manual,Berlina,1598.0,blanco,Valencia,C (verde),WebScraping,2025-05-16,202505.0,2025-05-16 15:56:26,https://www.coches.net/seat-ibiza-16-tdi-70kw-...


Los modelos también son distintos, pero lo que haremos es hacer una lista con todos los modelos del Dataset de Kaggle, que es más genérico, e identificar los modelos del WebScraping con estos. Si no es posible en alguno, dejaremos el modelo específico del WebScrapping.

Las siguientes transformaciones del dataset de coches de WebScraping serán incorporadas en el script de carga del scrapeo, para que los datos se carguen ya correctamente:

- modelo in modelos del dataset --> coge ese, si no, guarda el string.
- combustible en este formato: [Diésel, Gasolina, Híbrido enchufable, Eléctrico, Híbrido]
- color --> quedarse solo con el primero y en minúscula.




## Subida del DataFrame de Kaggle a la BBDD:

Ahora que tenemos el DataFrame de Kaggle en un formato compatible con el de los vehículos scrapeados, vamos a guardar los registros para que todos estén en la misma tabla.

Primero comprobamos el estado actual del DataFrame de Kaggle:

In [74]:
#Dataset Kaggle:

pd.set_option('display.max_columns', None)
df.head()

,pk_anuncio_id,marca,modelo,precio,combustible,anyo_vehiculo,kilometraje,potencia,num_puertas,num_plazas,tipo_cambio,tipo_vehiculo,cilindrada_motor,color,provincia,etiqueta_eco,origen_anuncio,fecha_publicacion,anyomes_publicacion,fecha_carga,url
0,1,SEAT,Toledo,950.0,Diésel,2000.0,227000,NaN,4,None,Manual,None,None,verde,Navarra,None,dataset kaggle,2020-12-18,202012,2021-01-15 00:00:00,e158ae0ca53119ca199c28c36b5c2fcd
1,2,CITROEN,C1,6200.0,Gasolina,2017.0,50071,82.0,5,None,Manual,None,None,blanco,Tenerife,None,dataset kaggle,2021-01-02,202101,2021-01-15 00:00:00,ff267ebb7e700246f47f84f3db660b4b
2,3,FORD,Transit Connect,7851.0,Diésel,2016.0,103000,100.0,4,None,Manual,None,None,blanco,Barcelona,None,dataset kaggle,2020-12-16,202012,2021-01-15 00:00:00,de4b02db28ea7786c622b969be10c7c7
3,4,VOLKSWAGEN,Caravelle,19426.0,Diésel,2014.0,120000,140.0,4,None,Manual,None,None,blanco,Navarra,None,dataset kaggle,2020-11-25,202011,2021-01-15 00:00:00,0449972a4d07594acf92e9a7dd28b39c
4,5,FORD,Transit,22850.0,Diésel,2017.0,107000,130.0,2,None,Manual,None,None,blanco,Sevilla,None,dataset kaggle,2021-01-12,202101,2021-01-15 00:00:00,12c4fa49bd4fdf23f19ecf396d3f02ef


In [76]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 35046 entries, 0 to 49999
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   pk_anuncio_id        35046 non-null  int64  
 1   marca                35046 non-null  object 
 2   modelo               35041 non-null  object 
 3   precio               35046 non-null  float64
 4   combustible          35028 non-null  object 
 5   anyo_vehiculo        35045 non-null  float64
 6   kilometraje          35046 non-null  int64  
 7   potencia             28266 non-null  float64
 8   num_puertas          35046 non-null  int64  
 9   num_plazas           0 non-null      object 
 10  tipo_cambio          34966 non-null  object 
 11  tipo_vehiculo        0 non-null      object 
 12  cilindrada_motor     0 non-null      object 
 13  color                34598 non-null  object 
 14  provincia            35042 non-null  object 
 15  etiqueta_eco         0 non-null      obje

Antes de cargar los datos, debemos eliminar los 5 registros que tienen el campo ```"modelo"``` = *null*, ya que es una constraint de la tabla creada.

In [ ]:
df = df.dropna(subset=['modelo'])


In [84]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 35041 entries, 0 to 49999
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   pk_anuncio_id        35041 non-null  int64  
 1   marca                35041 non-null  object 
 2   modelo               35041 non-null  object 
 3   precio               35041 non-null  float64
 4   combustible          35023 non-null  object 
 5   anyo_vehiculo        35040 non-null  float64
 6   kilometraje          35041 non-null  int64  
 7   potencia             28265 non-null  float64
 8   num_puertas          35041 non-null  int64  
 9   num_plazas           0 non-null      object 
 10  tipo_cambio          34961 non-null  object 
 11  tipo_vehiculo        0 non-null      object 
 12  cilindrada_motor     0 non-null      object 
 13  color                34594 non-null  object 
 14  provincia            35037 non-null  object 
 15  etiqueta_eco         0 non-null      obje

Ahora subimos los registros a la BBDD:

In [85]:
import sqlite3

con = sqlite3.connect("../include/db_vehiculos.db")

try:
    df.to_sql('TX_VEHICULOS_SEG_MANO', con, if_exists='append', index=False)
    print("Datos insertados con éxito")
except Exception as e:
    print("Error al insertar datos:", e)

con.close()

Error al insertar datos: UNIQUE constraint failed: TX_VEHICULOS_SEG_MANO.pk_anuncio_id


In [103]:
import pandas as pd
import sqlite3

con = sqlite3.connect("../include/db_vehiculos.db")

query = """
SELECT *
FROM TX_VEHICULOS_SEG_MANO
--where
    --origen_anuncio = 'WebScraping'
    --origen_anuncio = 'dataset kaggle	'
    --and marca not in ('BMW', 'VOLKSWAGEN', 'MERCEDES-BENZ', 'AUDI', 'PEUGEOT', 'FORD', 'RENAULT', 'OPEL', 'CITROEN', 'SEAT')
    --pk_anuncio_id in (1, 2, 3, 4)
order by
    fecha_carga desc
;
"""

df_db_new = pd.read_sql_query(query, con)

df_db_new.head()

,pk_anuncio_id,marca,modelo,precio,combustible,anyo_vehiculo,kilometraje,potencia,num_puertas,num_plazas,tipo_cambio,tipo_vehiculo,cilindrada_motor,color,provincia,etiqueta_eco,origen_anuncio,fecha_publicacion,anyomes_publicacion,fecha_carga,url
0,60579674,OPEL,Corsa 1.2 XEL Edition,12290.0,Gasolina,2022.0,58168,75.0,5,5.0,Manual,Berlina,1199.0,Rojo,Valencia,C (verde),WebScraping,2025-05-16,202505.0,2025-05-16 15:58:23,https://www.coches.net/opel-corsa-12-xel-55kw-...
1,60579673,OPEL,Corsa 1.2T XHL GSLine,14510.0,Gasolina,2022.0,18652,100.0,5,5.0,Manual,Berlina,1199.0,Azul,Valencia,C (verde),WebScraping,2025-05-16,202505.0,2025-05-16 15:57:56,https://www.coches.net/opel-corsa-12t-xhl-74kw...
2,60579672,OPEL,Corsa 1.2T XHL GSLine,13950.0,Gasolina,2022.0,27493,100.0,5,5.0,Manual,Berlina,1199.0,Gris / Plata,Valencia,C (verde),WebScraping,2025-05-16,202505.0,2025-05-16 15:57:34,https://www.coches.net/opel-corsa-12t-xhl-74kw...
3,60579678,OPEL,Corsa 1.2 XEL Edition,12290.0,Gasolina,2022.0,49466,75.0,5,5.0,Manual,Berlina,1199.0,Blanco,Valencia,C (verde),WebScraping,2025-05-16,202505.0,2025-05-16 15:56:49,https://www.coches.net/opel-corsa-12-xel-55kw-...
4,60580266,SEAT,Ibiza 1.6 TDI Style,11499.0,Diesel,2018.0,147508,95.0,5,5.0,Manual,Berlina,1598.0,Blanco,Valencia,C (verde),WebScraping,2025-05-16,202505.0,2025-05-16 15:56:26,https://www.coches.net/seat-ibiza-16-tdi-70kw-...


In [104]:
df_db_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35772 entries, 0 to 35771
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   pk_anuncio_id        35772 non-null  int64  
 1   marca                35772 non-null  object 
 2   modelo               35772 non-null  object 
 3   precio               35772 non-null  float64
 4   combustible          35741 non-null  object 
 5   anyo_vehiculo        35771 non-null  float64
 6   kilometraje          35772 non-null  int64  
 7   potencia             28993 non-null  float64
 8   num_puertas          35772 non-null  int64  
 9   num_plazas           728 non-null    float64
 10  tipo_cambio          35692 non-null  object 
 11  tipo_vehiculo        731 non-null    object 
 12  cilindrada_motor     719 non-null    float64
 13  color                35316 non-null  object 
 14  provincia            35768 non-null  object 
 15  etiqueta_eco         564 non-null   

Vemos que se han insertado correctamente los 35.041 registros del dataset de Kaggle en la base de datos, que se suman a los 731 que teníamos procedentes del WebScraping.

Una vez que ya tenemos los registros insertados, se implementarán las mejoras de formato comentadas en el script del WebScraping, para que coincidan con el mostrado. Después, ya realizaremos la limpieza de datos definitiva y entrenaremos el modelo en el Notebook: "ml_car_price_prediction.ipynb".

Vamos a guardar también este último DataFrame con la versión final de los datos sacados de Kaggle y el formato de nuestra tabla TX_VEHICULOS_SEG_MANO, por si tenemos que utilizarlo posteriormente.

In [ ]:
df.to_csv("../include/data/processed/df_kaggle_db.csv", index=False)

## Extra: cargar datos finales del DF de Kaggle en BBDD:

In [ ]:
import pandas as pd

df = pd.read_csv("../include/data/processed/df_kaggle_db.csv")

df.head()

,pk_anuncio_id,marca,modelo,precio,combustible,anyo_vehiculo,kilometraje,potencia,num_puertas,num_plazas,...,tipo_vehiculo,cilindrada_motor,color,provincia,etiqueta_eco,origen_anuncio,fecha_publicacion,anyomes_publicacion,fecha_carga,url
0,1,SEAT,Toledo,950.0,Diésel,2000.0,227000,NaN,4,NaN,...,NaN,NaN,verde,Navarra,NaN,dataset kaggle,2020-12-18,202012,2021-01-15 00:00:00,e158ae0ca53119ca199c28c36b5c2fcd
1,2,CITROEN,C1,6200.0,Gasolina,2017.0,50071,82.0,5,NaN,...,NaN,NaN,blanco,Tenerife,NaN,dataset kaggle,2021-01-02,202101,2021-01-15 00:00:00,ff267ebb7e700246f47f84f3db660b4b
2,3,FORD,Transit Connect,7851.0,Diésel,2016.0,103000,100.0,4,NaN,...,NaN,NaN,blanco,Barcelona,NaN,dataset kaggle,2020-12-16,202012,2021-01-15 00:00:00,de4b02db28ea7786c622b969be10c7c7
3,4,VOLKSWAGEN,Caravelle,19426.0,Diésel,2014.0,120000,140.0,4,NaN,...,NaN,NaN,blanco,Navarra,NaN,dataset kaggle,2020-11-25,202011,2021-01-15 00:00:00,0449972a4d07594acf92e9a7dd28b39c
4,5,FORD,Transit,22850.0,Diésel,2017.0,107000,130.0,2,NaN,...,NaN,NaN,blanco,Sevilla,NaN,dataset kaggle,2021-01-12,202101,2021-01-15 00:00:00,12c4fa49bd4fdf23f19ecf396d3f02ef


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35041 entries, 0 to 35040
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   pk_anuncio_id        35041 non-null  int64  
 1   marca                35041 non-null  object 
 2   modelo               35041 non-null  object 
 3   precio               35041 non-null  float64
 4   combustible          35023 non-null  object 
 5   anyo_vehiculo        35040 non-null  float64
 6   kilometraje          35041 non-null  int64  
 7   potencia             28265 non-null  float64
 8   num_puertas          35041 non-null  int64  
 9   num_plazas           0 non-null      float64
 10  tipo_cambio          34961 non-null  object 
 11  tipo_vehiculo        0 non-null      float64
 12  cilindrada_motor     0 non-null      float64
 13  color                34594 non-null  object 
 14  provincia            35037 non-null  object 
 15  etiqueta_eco         0 non-null     

Antes de cargarlos, vamos a quitarle también los acentos a los valores de la columna modelo y a ponerlos en minúscula:

In [53]:
from unidecode import unidecode

df["modelo"] = df["modelo"].str.lower().apply(unidecode)

In [54]:
df[df["marca"] == "SEAT"]["modelo"].value_counts()


modelo
ibiza              824
leon               800
arona              233
ateca              143
alhambra            99
toledo              97
altea               83
cordoba             77
altea xl            76
exeo                54
tarraco             48
mii                 20
marbella            14
arosa               13
inca                10
altea freetrack      4
malaga               2
terra                2
Name: count, dtype: int64

In [59]:
import sqlite3

con = sqlite3.connect("../include/db_vehiculos.db")

try:
    df.to_sql('TX_VEHICULOS_SEG_MANO', con, if_exists='append', index=False)
    print("Datos insertados con éxito")
except Exception as e:
    print("Error al insertar datos:", e)

con.close()

Datos insertados con éxito


In [60]:
import pandas as pd
import sqlite3

con = sqlite3.connect("../include/db_vehiculos.db")

query = """
SELECT *
FROM TX_VEHICULOS_SEG_MANO
where
    --origen_anuncio = 'WebScraping'
    origen_anuncio = 'dataset kaggle'
    --marca not in ('BMW', 'VOLKSWAGEN', 'MERCEDES-BENZ', 'AUDI', 'PEUGEOT', 'FORD', 'RENAULT', 'OPEL', 'CITROEN', 'SEAT')
    --pk_anuncio_id in (1, 2, 3, 4)
order by
    fecha_carga desc
;
"""

df_db_new = pd.read_sql_query(query, con)

df_db_new.head()

,pk_anuncio_id,marca,modelo,precio,combustible,anyo_vehiculo,kilometraje,potencia,num_puertas,num_plazas,...,tipo_vehiculo,cilindrada_motor,color,provincia,etiqueta_eco,origen_anuncio,fecha_publicacion,anyomes_publicacion,fecha_carga,url
0,1,SEAT,toledo,950.0,Diésel,2000.0,227000,NaN,4,None,...,None,None,verde,Navarra,None,dataset kaggle,2020-12-18,202012,2021-01-15 00:00:00,e158ae0ca53119ca199c28c36b5c2fcd
1,2,CITROEN,c1,6200.0,Gasolina,2017.0,50071,82.0,5,None,...,None,None,blanco,Tenerife,None,dataset kaggle,2021-01-02,202101,2021-01-15 00:00:00,ff267ebb7e700246f47f84f3db660b4b
2,3,FORD,transit connect,7851.0,Diésel,2016.0,103000,100.0,4,None,...,None,None,blanco,Barcelona,None,dataset kaggle,2020-12-16,202012,2021-01-15 00:00:00,de4b02db28ea7786c622b969be10c7c7
3,4,VOLKSWAGEN,caravelle,19426.0,Diésel,2014.0,120000,140.0,4,None,...,None,None,blanco,Navarra,None,dataset kaggle,2020-11-25,202011,2021-01-15 00:00:00,0449972a4d07594acf92e9a7dd28b39c
4,5,FORD,transit,22850.0,Diésel,2017.0,107000,130.0,2,None,...,None,None,blanco,Sevilla,None,dataset kaggle,2021-01-12,202101,2021-01-15 00:00:00,12c4fa49bd4fdf23f19ecf396d3f02ef


In [56]:
df_db_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35041 entries, 0 to 35040
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   pk_anuncio_id        35041 non-null  int64  
 1   marca                35041 non-null  object 
 2   modelo               35041 non-null  object 
 3   precio               35041 non-null  float64
 4   combustible          35023 non-null  object 
 5   anyo_vehiculo        35040 non-null  float64
 6   kilometraje          35041 non-null  int64  
 7   potencia             28265 non-null  float64
 8   num_puertas          35041 non-null  int64  
 9   num_plazas           0 non-null      object 
 10  tipo_cambio          34961 non-null  object 
 11  tipo_vehiculo        0 non-null      object 
 12  cilindrada_motor     0 non-null      object 
 13  color                34594 non-null  object 
 14  provincia            35037 non-null  object 
 15  etiqueta_eco         0 non-null     

Volvemos a guardar en .csv con el nuevo cambio:

In [63]:
df_db_new.to_csv("../include/data/processed/df_kaggle_final_db.csv", index=False)

prueba:

In [61]:
import pandas as pd
import sqlite3

con = sqlite3.connect("../include/db_vehiculos.db")

query = """
SELECT marca, modelo
FROM TX_VEHICULOS_SEG_MANO
--where
    --origen_anuncio = 'dataset kaggle'
    --origen_anuncio = 'WebScraping'
group by
    marca, modelo
;
"""

df_db_modelo = pd.read_sql_query(query, con)

df_db_modelo.head()

,marca,modelo
0,AUDI,100
1,AUDI,80
2,AUDI,90
3,AUDI,a1
4,AUDI,a2


In [62]:
df_db_modelo["modelo"].value_counts()

modelo
transporter    2
100            1
208            1
4008           1
4007           1
              ..
galaxy         1
fusion         1
focus c-max    1
focus          1
vento          1
Name: count, Length: 390, dtype: int64